# importação de backups

In [16]:
import pyodbc as PDB  # Drive para conexão com banco de dados
import pandas  as pd
import sys
import os
import traceback
import json
import re

In [2]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, __version__

In [118]:
print("Azure Blob Storage v" + __version__ + " - Python quickstart sample")

Azure Blob Storage v12.8.0 - Python quickstart sample


## Importa o arquivo com as configurações


In [3]:
datasource = pd.read_excel(r'D:\Ondrive\OneDrive - Implanta Informatica\Migração arquivos Anexos\ClientesComESemComfiguracaoArquivosAnexos.xlsx',sheet_name='registros')

In [4]:
datasource.reset_index()

,index,Cliente,Categoria,object_id,Nome Schema,Nome Tabela,Bytes/Row,Quantidade Registros,MigracaoDiaria,QuantideDias,...,TotalMB,Total MB Por Schema,TotalAlocadoMB,TotalUsadoMB,UtilizaAzure,StorageAccount,ContainerAzureStorageArquivosAnexos,CaminhoCacheLocalAzureStorageArquivosAnexos,ConnectionStringAzureStorageArquivosAnexos,HorarioMigracaoArquivosAnexosParaAzureStorage
0,0,cau-ac.implanta.net.br,cau,62479847,Sistema,ArquivosAnexos,5410.0,3.699,500,1,...,19.0,19.0,19.00 MB,17 MB,SIM,improblob01,cau-ac,D:\temp_blob_azure\cau-ac\,DefaultEndpointsProtocol=https;AccountName=imp...,23:00:00
1,1,cau-al.implanta.net.br,cau,2086766987,Sistema,ArquivosAnexos,211995.0,2.936,500,6,...,593.0,593.0,593.00 MB,588 MB,NAO,improblob01,cau-al,D:\temp_blob_azure\cau-al\,DefaultEndpointsProtocol=https;AccountName=imp...,23:00:00
2,2,cau-am.implanta.net.br,cau,1115007599,Sistema,ArquivosAnexos,240190.0,3.799,500,8,...,870.0,870.0,870.00 MB,863 MB,NAO,improblob01,cau-am,D:\temp_blob_azure\cau-am\,DefaultEndpointsProtocol=https;AccountName=imp...,21:00:00
3,3,cau-ap.implanta.net.br,cau,378484427,Sistema,ArquivosAnexos,191347.0,4.018,500,9,...,733.0,733.0,733.00 MB,731 MB,NAO,improblob01,cau-ap,D:\temp_blob_azure\cau-ap\,DefaultEndpointsProtocol=https;AccountName=imp...,23:00:00
4,4,cau-ba.implanta.net.br,cau,378484427,Sistema,ArquivosAnexos,401356.0,1.284,500,3,...,491.0,491.0,491.00 MB,489 MB,NAO,improblob01,cau-ba,D:\temp_blob_azure\cau-ba\,DefaultEndpointsProtocol=https;AccountName=imp...,22:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,380,oab-pi.implanta.net.br,oab,538484997,Sistema,ArquivosAnexos,116593.0,43,500,1,...,4.0,4.0,4.00 MB,4 MB,NAO,improblob03,oab-pi,D:\temp_blob_azure\oab-pi\,DefaultEndpointsProtocol=https;AccountName=imp...,02:00:00
381,381,oab-rn.implanta.net.br,oab,538484997,Sistema,ArquivosAnexos,1378183.0,17,500,1,...,22.0,22.0,22.00 MB,22 MB,NAO,improblob03,oab-rn,D:\temp_blob_azure\oab-rn\,DefaultEndpointsProtocol=https;AccountName=imp...,03:00:00
382,382,oab-rr.implanta.net.br,oab,538484997,Sistema,ArquivosAnexos,147456.0,2,500,1,...,0.0,0.0,0.00 MB,0 MB,NAO,improblob03,oab-rr,D:\temp_blob_azure\oab-rr\,DefaultEndpointsProtocol=https;AccountName=imp...,23:00:00
383,383,oab-rs.implanta.net.br,oab,1190672431,Sistema,ArquivosAnexos,219649.0,65.016,500,131,...,13619.0,13619.0,13619.00 MB,13602 MB,NAO,improblob03,oab-rs,D:\temp_blob_azure\oab-rs\,DefaultEndpointsProtocol=https;AccountName=imp...,02:00:00


## Planilha com as configuracoes dos bancos de dados


In [5]:
datasource[0:1]

,Cliente,Categoria,object_id,Nome Schema,Nome Tabela,Bytes/Row,Quantidade Registros,MigracaoDiaria,QuantideDias,(No column name),...,TotalMB,Total MB Por Schema,TotalAlocadoMB,TotalUsadoMB,UtilizaAzure,StorageAccount,ContainerAzureStorageArquivosAnexos,CaminhoCacheLocalAzureStorageArquivosAnexos,ConnectionStringAzureStorageArquivosAnexos,HorarioMigracaoArquivosAnexosParaAzureStorage
0,cau-ac.implanta.net.br,cau,62479847,Sistema,ArquivosAnexos,5410.0,3.699,500,1,==>,...,19.0,19.0,19.00 MB,17 MB,SIM,improblob01,cau-ac,D:\temp_blob_azure\cau-ac\,DefaultEndpointsProtocol=https;AccountName=imp...,23:00:00


# Chave dos containers
improblob01

In [75]:
for index , row in datasource.iterrows():
    database = row["Cliente"]
    categoria = row["Categoria"]
    migracaoDiaria = row["MigracaoDiaria"]
    totalArquivosNoBanco = row["TotalArquivosNoBanco"]
    UtilizaAzure = row["UtilizaAzure"]
    storageAccount =  row["StorageAccount"]
    container =row["ContainerAzureStorageArquivosAnexos"]
    horarioMigracao =row["HorarioMigracaoArquivosAnexosParaAzureStorage"]
    connect_str_to_storage = row["ConnectionStringAzureStorageArquivosAnexos"]

    blob_service_client = BlobServiceClient.from_connection_string(connect_str_to_storage)

    existContainer = blob_service_client.get_container_client(container).exists()

    
    if(existContainer ==False):
        blob_service_client.create_container(container)
        print('database:',database,' storageAccount:',storageAccount,' Container Criado',container)
    else:
         print('database:',database,' storageAccount:',storageAccount,' Container já existe',container)


    

    

database: cau-ac.implanta.net.br  storageAccount: improblob01  Container já existe cau-ac
database: cau-al.implanta.net.br  storageAccount: improblob01  Container já existe cau-al
database: cau-am.implanta.net.br  storageAccount: improblob01  Container já existe cau-am
database: cau-ap.implanta.net.br  storageAccount: improblob01  Container já existe cau-ap
database: cau-ba.implanta.net.br  storageAccount: improblob01  Container já existe cau-ba
database: cau-br.implanta.net.br  storageAccount: improblob01  Container já existe cau-br
database: cau-ce.implanta.net.br  storageAccount: improblob01  Container já existe cau-ce
database: cau-df.implanta.net.br  storageAccount: improblob01  Container já existe cau-df
database: cau-es.implanta.net.br  storageAccount: improblob01  Container já existe cau-es
database: cau-go.implanta.net.br  storageAccount: improblob01  Container já existe cau-go
database: cau-ma.implanta.net.br  storageAccount: improblob01  Container já existe cau-ma
database: 

## recupera os dados finais para gerar o excel

In [7]:
dados = datasource[['Cliente','Categoria','MigracaoDiaria','UtilizaAzure','StorageAccount','ContainerAzureStorageArquivosAnexos', 'CaminhoCacheLocalAzureStorageArquivosAnexos','ConnectionStringAzureStorageArquivosAnexos','HorarioMigracaoArquivosAnexosParaAzureStorage']]

## Gera o excel com as configurações 

In [123]:
dados.to_excel('D:\Ondrive\OneDrive - Implanta Informatica\Migração arquivos Anexos\ConfiguracaoFinalStorageAccounts.xlsx')

## cria um dataset com as seguintes categorias a ser configuradas
#cau
#core
#coren
#crea
#cra
#cro

In [9]:
filter_caregorias = dados.query("Categoria in('cau','core','coren','crea','cra','cro') and  UtilizaAzure =='NAO'")[['Cliente','Categoria','MigracaoDiaria','UtilizaAzure','StorageAccount','ContainerAzureStorageArquivosAnexos','CaminhoCacheLocalAzureStorageArquivosAnexos','ConnectionStringAzureStorageArquivosAnexos','HorarioMigracaoArquivosAnexosParaAzureStorage']]

## mostra os 5 primeiros

In [10]:
filter_caregorias.head(5)

,Cliente,Categoria,MigracaoDiaria,UtilizaAzure,StorageAccount,ContainerAzureStorageArquivosAnexos,CaminhoCacheLocalAzureStorageArquivosAnexos,ConnectionStringAzureStorageArquivosAnexos,HorarioMigracaoArquivosAnexosParaAzureStorage
1,cau-al.implanta.net.br,cau,500,NAO,improblob01,cau-al,D:\temp_blob_azure\cau-al\,DefaultEndpointsProtocol=https;AccountName=imp...,23:00:00
2,cau-am.implanta.net.br,cau,500,NAO,improblob01,cau-am,D:\temp_blob_azure\cau-am\,DefaultEndpointsProtocol=https;AccountName=imp...,21:00:00
3,cau-ap.implanta.net.br,cau,500,NAO,improblob01,cau-ap,D:\temp_blob_azure\cau-ap\,DefaultEndpointsProtocol=https;AccountName=imp...,23:00:00
4,cau-ba.implanta.net.br,cau,500,NAO,improblob01,cau-ba,D:\temp_blob_azure\cau-ba\,DefaultEndpointsProtocol=https;AccountName=imp...,22:00:00
5,cau-br.implanta.net.br,cau,500,NAO,improblob01,cau-br,D:\temp_blob_azure\cau-br\,DefaultEndpointsProtocol=https;AccountName=imp...,01:00:00


## mostra os 5 ultimos

In [11]:
filter_caregorias.tail(5)

,Cliente,Categoria,MigracaoDiaria,UtilizaAzure,StorageAccount,ContainerAzureStorageArquivosAnexos,CaminhoCacheLocalAzureStorageArquivosAnexos,ConnectionStringAzureStorageArquivosAnexos,HorarioMigracaoArquivosAnexosParaAzureStorage
236,crea-rs.implanta.net.br,crea,500,NAO,improblob03,crea-rs,D:\temp_blob_azure\crea-rs\,DefaultEndpointsProtocol=https;AccountName=imp...,23:00:00
237,crea-sc.implanta.net.br,crea,500,NAO,improblob03,crea-sc,D:\temp_blob_azure\crea-sc\,DefaultEndpointsProtocol=https;AccountName=imp...,01:00:00
238,crea-se.implanta.net.br,crea,500,NAO,improblob03,crea-se,D:\temp_blob_azure\crea-se\,DefaultEndpointsProtocol=https;AccountName=imp...,22:00:00
239,crea-sp.implanta.net.br,crea,500,NAO,improblob03,crea-sp,D:\temp_blob_azure\crea-sp\,DefaultEndpointsProtocol=https;AccountName=imp...,19:00:00
240,crea-to.implanta.net.br,crea,500,NAO,improblob03,crea-to,D:\temp_blob_azure\crea-to\,DefaultEndpointsProtocol=https;AccountName=imp...,19:00:00


## Conecta no banco de dados para alterar as configurações conforme a planilha

In [53]:
server = 'rgprd-sqlsrv-prd01.database.windows.net'
database = 'cau-ac.implanta.net.br'
username = 'wesley.neves@implantainformatica.com.br'
password = 'alice_96086512'
driver= '{ODBC Driver 17 for SQL Server}'

In [54]:
def get_conection(driver,server,database,username,password):
    _conexao = PDB.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password +';Authentication=ActiveDirectoryPassword')
    return _conexao

In [56]:
filter_caregorias.query("Cliente =='cau-al.implanta.net.br'")

,Cliente,Categoria,MigracaoDiaria,UtilizaAzure,StorageAccount,ContainerAzureStorageArquivosAnexos,CaminhoCacheLocalAzureStorageArquivosAnexos,ConnectionStringAzureStorageArquivosAnexos,HorarioMigracaoArquivosAnexosParaAzureStorage
1,cau-al.implanta.net.br,cau,500,NAO,improblob01,cau-al,D:\temp_blob_azure\cau-al\,DefaultEndpointsProtocol=https;AccountName=imp...,23:00:00


## script base para alteração das configurações

In [57]:
base_script_update = "UPDATE C SET C.Valor ='{0}' FROM  Sistema.Configuracoes C WHERE C.Configuracao ='{1}'"
print(base_script_update)

UPDATE C SET C.Valor ='{0}' FROM  Sistema.Configuracoes C WHERE C.Configuracao ='{1}'


In [58]:
base_script_insert = "INSERT INTO Sistema.Configuracoes VALUES(NEWID(), '00000000-0000-0000-0000-000000000000', 'Global', '{0}', '{1}', 0);"

print(base_script_insert)


INSERT INTO Sistema.Configuracoes VALUES(NEWID(), '00000000-0000-0000-0000-000000000000', 'Global', '{0}', '{1}', 0);


In [59]:
base_script_select ="select top 1 valor from sistema.Configuracoes where Configuracao ='{0}'"
print(base_script_select.format('QuantidadeDeArquivosMigradosPorExecucao'))

select top 1 valor from sistema.Configuracoes where Configuracao ='QuantidadeDeArquivosMigradosPorExecucao'


In [110]:
filter_caregorias.query("Categoria =='cau'")

,Cliente,Categoria,MigracaoDiaria,UtilizaAzure,StorageAccount,ContainerAzureStorageArquivosAnexos,CaminhoCacheLocalAzureStorageArquivosAnexos,ConnectionStringAzureStorageArquivosAnexos,HorarioMigracaoArquivosAnexosParaAzureStorage
1,cau-al.implanta.net.br,cau,500,NAO,improblob01,cau-al,D:\temp_blob_azure\cau-al\,DefaultEndpointsProtocol=https;AccountName=imp...,23:00:00
2,cau-am.implanta.net.br,cau,500,NAO,improblob01,cau-am,D:\temp_blob_azure\cau-am\,DefaultEndpointsProtocol=https;AccountName=imp...,21:00:00
3,cau-ap.implanta.net.br,cau,500,NAO,improblob01,cau-ap,D:\temp_blob_azure\cau-ap\,DefaultEndpointsProtocol=https;AccountName=imp...,23:00:00
4,cau-ba.implanta.net.br,cau,500,NAO,improblob01,cau-ba,D:\temp_blob_azure\cau-ba\,DefaultEndpointsProtocol=https;AccountName=imp...,22:00:00
5,cau-br.implanta.net.br,cau,500,NAO,improblob01,cau-br,D:\temp_blob_azure\cau-br\,DefaultEndpointsProtocol=https;AccountName=imp...,01:00:00
6,cau-ce.implanta.net.br,cau,500,NAO,improblob01,cau-ce,D:\temp_blob_azure\cau-ce\,DefaultEndpointsProtocol=https;AccountName=imp...,22:00:00
7,cau-df.implanta.net.br,cau,500,NAO,improblob01,cau-df,D:\temp_blob_azure\cau-df\,DefaultEndpointsProtocol=https;AccountName=imp...,24:00:00
8,cau-es.implanta.net.br,cau,500,NAO,improblob01,cau-es,D:\temp_blob_azure\cau-es\,DefaultEndpointsProtocol=https;AccountName=imp...,24:00:00
9,cau-go.implanta.net.br,cau,500,NAO,improblob01,cau-go,D:\temp_blob_azure\cau-go\,DefaultEndpointsProtocol=https;AccountName=imp...,19:00:00
10,cau-ma.implanta.net.br,cau,500,NAO,improblob01,cau-ma,D:\temp_blob_azure\cau-ma\,DefaultEndpointsProtocol=https;AccountName=imp...,02:00:00


## categorias configuradas

In [ ]:
dados

In [130]:
filter_caregorias['Categoria'].unique()

array(['cau', 'cra', 'cro', 'core', 'coren', 'crea'], dtype=object)

In [168]:
for index , row in filter_caregorias.iterrows():
    database = row["Cliente"]
    categoria = row["Categoria"]
    migracaoDiaria = row["MigracaoDiaria"]
    UtilizaAzure = row["UtilizaAzure"]
    
    storageAccount =  row["StorageAccount"]
    container =row["ContainerAzureStorageArquivosAnexos"]
    horarioMigracao =row["HorarioMigracaoArquivosAnexosParaAzureStorage"]
    connect_str_to_storage = row["ConnectionStringAzureStorageArquivosAnexos"]
    cLocalAzureStorageArquivosAnexos =row["CaminhoCacheLocalAzureStorageArquivosAnexos"]

    
    if(categoria =='crea'):
        blob_service_client = BlobServiceClient.from_connection_string(connect_str_to_storage)
        existContainer = blob_service_client.get_container_client(container).exists()

        
        if(existContainer ==True):
            _con = get_conection(driver,server,database,username,password)
            cursor = _con.cursor()

            #Valida se o cliente já está configurado
            query = base_script_select.format('UsaAzureStorageArquivosAnexos')
            cursor.execute(query)

            clienteConfigurado = cursor.fetchone()

            # if(clienteConfigurado is not None):
            #     if(clienteConfigurado[0] == 'true'):
            #         print(database,'Já configurado')
            #         _con.commit()
            #         cursor.close()
            #         continue



            # configura  QuantidadeDeArquivosMigradosPorExecucao
            print('configurando ',database)
            query = base_script_select.format('QuantidadeDeArquivosMigradosPorExecucao')
            cursor.execute(query)

            existe_QuantidadeDeArquivosMigradosPorExecucao = cursor.fetchone()
            
            if(existe_QuantidadeDeArquivosMigradosPorExecucao is not None):
                if(existe_QuantidadeDeArquivosMigradosPorExecucao[0] != '500'):
                    query_updateConfiguracao =base_script_update.format(500,'QuantidadeDeArquivosMigradosPorExecucao')
                    print(query_updateConfiguracao)
                    cursor.execute(query_updateConfiguracao)
            else:
                query_insert_configuracao = base_script_insert.format('QuantidadeDeArquivosMigradosPorExecucao',500)
                print(query_insert_configuracao)
                cursor.execute(query_insert_configuracao)

            
            #configura  HorarioMigracaoArquivosAnexosParaAzureStorage
            query = base_script_select.format('HorarioMigracaoArquivosAnexosParaAzureStorage')
            cursor.execute(query)

            existe_HorarioMigracaoArquivosAnexosParaAzureStorage =cursor.fetchone()
            
            if(existe_HorarioMigracaoArquivosAnexosParaAzureStorage is None):
                query_insert_configuracao = base_script_insert.format('HorarioMigracaoArquivosAnexosParaAzureStorage',horarioMigracao)
                cursor.execute(query_insert_configuracao)
                print(query_insert_configuracao)
            
            

             #configura  CaminhoCacheLocalAzureStorageArquivosAnexos
            query = base_script_select.format('CaminhoCacheLocalAzureStorageArquivosAnexos')
            cursor.execute(query)

            existe_CaminhoCacheLocalAzureStorageArquivosAnexos = cursor.fetchone()
            if(existe_CaminhoCacheLocalAzureStorageArquivosAnexos is None):
                query_insert_configuracao = base_script_insert.format('CaminhoCacheLocalAzureStorageArquivosAnexos',
            cLocalAzureStorageArquivosAnexos)
                cursor.execute(query_insert_configuracao)
                print(query_insert_configuracao)


            #configura  ConnectionStringAzureStorageArquivosAnexos
            query = base_script_select.format('ConnectionStringAzureStorageArquivosAnexos')
            cursor.execute(query)

            existe_ConnectionStringAzureStorageArquivosAnexos = cursor.fetchone()
            if(existe_ConnectionStringAzureStorageArquivosAnexos is None):
                query_insert_configuracao = base_script_insert.format('ConnectionStringAzureStorageArquivosAnexos',connect_str_to_storage)
                cursor.execute(query_insert_configuracao)
                print(query_insert_configuracao)
            
            #configura  ConnectionStringAzureStorageArquivosAnexos
            query = base_script_select.format('ContainerAzureStorageArquivosAnexos')
            cursor.execute(query)

            existe_ContainerAzureStorageArquivosAnexos = cursor.fetchone()
            if(existe_ContainerAzureStorageArquivosAnexos is None):
                query_insert_configuracao = base_script_insert.format('ContainerAzureStorageArquivosAnexos',container)
                cursor.execute(query_insert_configuracao)
                print(query_insert_configuracao)
            

            #configura  UsaAzureStorageArquivosAnexos
            query = base_script_select.format('UsaAzureStorageArquivosAnexos')
            cursor.execute(query)

            existe_UsaAzureStorageArquivosAnexos = cursor.fetchone()
            if(existe_UsaAzureStorageArquivosAnexos is None):
                query_insert_configuracao = base_script_insert.format('UsaAzureStorageArquivosAnexos','true')
                cursor.execute(query_insert_configuracao)
                print(query_insert_configuracao)
            
            else:
                if(existe_UsaAzureStorageArquivosAnexos[0] =='false'):
                    query_update_configuracao = base_script_update.format('true','UsaAzureStorageArquivosAnexos')
                    cursor.execute(query_update_configuracao)
                    print(query_update_configuracao)
            




            _con.commit()
            cursor.close()

                
                
                
            
            


            
       
        

        

configurando  crea-ac.implanta.net.br
UPDATE C SET C.Valor ='500' FROM  Sistema.Configuracoes C WHERE C.Configuracao ='QuantidadeDeArquivosMigradosPorExecucao'
INSERT INTO Sistema.Configuracoes VALUES(NEWID(), '00000000-0000-0000-0000-000000000000', 'Global', 'CaminhoCacheLocalAzureStorageArquivosAnexos', 'D:\temp_blob_azure\crea-ac\', 0);
INSERT INTO Sistema.Configuracoes VALUES(NEWID(), '00000000-0000-0000-0000-000000000000', 'Global', 'ConnectionStringAzureStorageArquivosAnexos', 'DefaultEndpointsProtocol=https;AccountName=improblob03;AccountKey=WEA4YzMZHaeDg0TBLXQje+EkZanSNVyPxKvmYKGQeVO/Blo9bsjlI4yjpI7DivAMuK6KPvoRBHq5MRZzqDdJEA==;EndpointSuffix=core.windows.net', 0);
INSERT INTO Sistema.Configuracoes VALUES(NEWID(), '00000000-0000-0000-0000-000000000000', 'Global', 'ContainerAzureStorageArquivosAnexos', 'crea-ac', 0);
UPDATE C SET C.Valor ='true' FROM  Sistema.Configuracoes C WHERE C.Configuracao ='UsaAzureStorageArquivosAnexos'
configurando  crea-al.implanta.net.br
UPDATE C SET 

## Validação  das alterações

In [171]:
sql_query_Validation = ("SELECT COUNT(1) FROM Sistema.Configuracoes AS C WHERE "
       "C.Configuracao IN ( "
       "'QuantidadeDeArquivosMigradosPorExecucao', 'CaminhoCacheLocalAzureStorageArquivosAnexos',"
       "'ConnectionStringAzureStorageArquivosAnexos', 'ContainerAzureStorageArquivosAnexos', "
       "'HorarioMigracaoArquivosAnexosParaAzureStorage', 'UsaAzureStorageArquivosAnexos') ")
     

In [172]:
for index , row in filter_caregorias.iterrows():
    database = row["Cliente"]
    categoria = row["Categoria"]
    migracaoDiaria = row["MigracaoDiaria"]
    UtilizaAzure = row["UtilizaAzure"]
    
    storageAccount =  row["StorageAccount"]
    container =row["ContainerAzureStorageArquivosAnexos"]
    horarioMigracao =row["HorarioMigracaoArquivosAnexosParaAzureStorage"]
    connect_str_to_storage = row["ConnectionStringAzureStorageArquivosAnexos"]
    cLocalAzureStorageArquivosAnexos =row["CaminhoCacheLocalAzureStorageArquivosAnexos"]

    
    
    blob_service_client = BlobServiceClient.from_connection_string(connect_str_to_storage)
    existContainer = blob_service_client.get_container_client(container).exists()

    if(existContainer ==False):
        print('Container não configurado:',container)

    if(existContainer ==True):
        _con = get_conection(driver,server,database,username,password)
        cursor = _con.cursor()

        cursor.execute(sql_query_Validation)

        validate = cursor.fetchone()
        if(validate[0] != 6):
            print(database, 'Com erro')


        _con.commit()
        cursor.close()


